In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten
import pandas as pd
import numpy as np
import os

In [2]:
# Load dataset
df = pd.read_csv("../Training_Data/balanced_soldier_data.csv")

In [4]:
df.head()

,Temperature,Moisture,Water_Content,SpO2,Fatigue,Drowsiness,Stress,Heart_Rate,Respiration_Rate,Systolic_BP,Diastolic_BP,Efficiency
0,35.383229,65.701296,74.896995,96.144655,0.208223,0.078256,0.167958,66.431073,15.907304,119,78,0.726903
1,36.028752,69.182477,87.289935,99.285356,0.186550,0.138735,0.008335,82.207135,17.644151,113,76,0.775944
2,37.004390,38.142130,51.690862,88.401253,0.795877,0.860479,0.821897,115.268466,25.442539,137,88,0.216759
3,35.926560,56.353209,86.145719,97.281401,0.074097,0.278761,0.125591,86.724807,15.603473,117,70,0.719477
4,36.231738,46.817432,48.037744,89.566148,0.779676,0.624617,0.829127,118.251208,23.152386,135,85,0.247567


In [5]:
df["Squad Efficiency"] = df.groupby(df.index // 10)["Efficiency"].transform("mean")

In [6]:
# Select relevant features
features = [
    "Temperature",
    "Moisture",
    "Water_Content",
    "SpO2",
    "Fatigue",
    "Drowsiness",
    "Stress",
    "Heart_Rate",
    "Respiration_Rate",
    "Systolic_BP",
    "Diastolic_BP",
    "Squad Efficiency",
]
target = "Strike Success Probability"  # This will be our label

In [7]:
# Generate synthetic labels (for demonstration)
df[target] = df["Squad Efficiency"].apply(
    lambda x: np.random.uniform(0, 1) if x > 0.6 else np.random.uniform(0, 0.5)
)

In [8]:
# Normalize the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [9]:
X = df[features].values
y = df[target].values

In [10]:
# Reshape input for LSTM (Samples, Time Steps, Features)
X = X.reshape((X.shape[0] // 10, 10, X.shape[1]))  # Grouping 10 soldiers per second
y = y[::10]  # One label per second

In [11]:
# Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [12]:
def build_model():
    # Build the Deep Learning Model
    model = Sequential(
        [
            LSTM(64, return_sequences=True, input_shape=(10, len(features))),
            Dropout(0.2),
            LSTM(32, return_sequences=False),
            Dropout(0.2),
            Dense(16, activation="relu"),
            Dense(8, activation="relu"),
            Dense(1, activation="sigmoid"),  # Output between 0 and 1
        ]
    )
    return model

In [13]:
model = build_model()

c:\Users\Himanshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [15]:
# Train the model
history = model.fit(
    X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test)
)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.0000e+00 - loss: 0.6680 - val_accuracy: 0.0000e+00 - val_loss: 0.5657
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0000e+00 - loss: 0.5842 - val_accuracy: 0.0000e+00 - val_loss: 0.5679
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 0.5655 - val_accuracy: 0.0000e+00 - val_loss: 0.5643
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 - loss: 0.5767 - val_accuracy: 0.0000e+00 - val_loss: 0.5654
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 0.5812 - val_accuracy: 0.0000e+00 - val_loss: 0.5645
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 0.5745 - val_accuracy: 0.0000e+00 - val_loss: 0.5646
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 0.5783 - val_accuracy: 0.0000e+00 - val_loss: 0.5647
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - ac

In [19]:
Trained_Model_Path = "../../backend/Trained_Models"

In [20]:
# Save the model
model.save(os.path.join(Trained_Model_Path,"Surgical_Model.keras"))

In [21]:
# Test the model
test_prediction = model.predict(X_test[:5])
print("Predicted Success Chances:", test_prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
Predicted Success Chances: [[0.2612658 ]
 [0.2579996 ]
 [0.25738484]
 [0.25568652]
 [0.25339988]]


In [22]:
import joblib

# Save the fitted scaler
joblib.dump(scaler, os.path.join(Trained_Model_Path,"Surgical_Scaler.pkl"))
print("Scaler saved successfully.")

Scaler saved successfully.
